# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py F1AP.docx 'keyword'" for F1AP and "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py NGAP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: NAS messages silently discarded for unknown UE contexts without AMF notification.
Repro Steps:
Establish and release UE context (AMF-UE-NGAP-ID=100, RAN-UE-NGAP-ID=200).
AMF sends DOWNLINK NAS TRANSPORT for the released context with AMF-UE-NGAP-ID=100, RAN-UE-NGAP-ID=200.
OAI NG-RAN logs the error locally but does not send NAS NON DELIVERY INDICATION to AMF.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "OAI NG-RAN logs an error locally but fails to send NAS NON DELIVERY INDICATION to AMF when receiving DOWNLINK NAS TRANSPORT for a released UE context (AMF-UE-NGAP-ID=100, RAN-UE-NGAP-ID=200).",
    "interface_guess": ["NGAP"],
    "procedure_guess": ["Downlink NAS Transport", "NAS Non Delivery Indication"],
    "components_involved": ["gNB-CU", "AMF"],
    "key_ids": {
      "cu_ue_f1ap_id": "200"
    },
    "signals_or_timers": []
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": ["DOWNLINK NAS TRANSPORT"],
      "title": ["8.6.2 Downlink NAS Transport"]
    },
    {
      "Keywords": ["NAS NON DELIVERY INDICATION"],
      "title": ["8.6.4 NAS Non Delivery Indication"]
    },
    {
      "Keywords": ["UE Context Release"],
      "title": ["8.3.3 UE Context Release"]
    }
  ],
  "expected_behaviour": "When the NG-RAN node receives a DOWNLINK NAS TRANSPORT message but cannot deliver the NAS PDU to the UE (e.g. because the UE is unknown or in an incompatible state), it should report this failure to the AMF using the NAS NON DELIVERY INDICATION procedure."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "openair3/NGAP/ngap_gNB_nas_procedures.c",
      "function_name": "ngap_gNB_nas_non_delivery_ind",
      "reason": "Handler for generating NAS Non Delivery Indication which is failing to send."
    },
    {
      "path": "openair2/RRC/NR/rrc_gNB_NGAP.c",
      "function_name": "rrc_gNB_process_NGAP_DOWNLINK_NAS",
      "reason": "RRC handler receiving DOWNLINK NAS, detecting unknown UE, and triggering the indication."
    },
    {
      "path": "openair3/NGAP/ngap_gNB_nas_procedures.c",
      "function_name": "ngap_gNB_handle_nas_downlink",
      "reason": "Initial NGAP handler for Downlink NAS Transport from AMF."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "code",
      "source": "openair3/NGAP/ngap_gNB_nas_procedures.c",
      "location": "490:495",
      "text": "if ((ue_context_p = ngap_get_ue_context(ngap_nas_non_delivery_ind->gNB_ue_ngap_id)) == NULL) {\n  /* The context for this gNB ue ngap id doesn't exist in the map of gNB UEs */\n  NGAP_WARN(\"Failed to find ue context associated with gNB ue ngap id: %08x\\n\", ngap_nas_non_delivery_ind->gNB_ue_ngap_id);\n  return -1;\n}"
    },
    {
      "kind": "code",
      "source": "openair2/RRC/NR/rrc_gNB_NGAP.c",
      "location": "706:716",
      "text": "if (ue_context_p == NULL) {\n  /* Can not associate this message to an UE index, send a failure to NGAP and discard it! */\n  LOG_W(NR_RRC, \"[gNB %ld] In NGAP_DOWNLINK_NAS: unknown UE from NGAP ids (%u)\\n\", instance, req->gNB_ue_ngap_id);\n  msg_fail_p = itti_alloc_new_message(TASK_RRC_GNB, 0, NGAP_NAS_NON_DELIVERY_IND);\n  ...\n  itti_send_msg_to_task(TASK_NGAP, instance, msg_fail_p);\n  return (-1);\n}"
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {
      "from": "AMF",
      "to": "NGAP",
      "message": "DOWNLINK NAS TRANSPORT",
      "precond": "UE Context Released at gNB"
    },
    {
      "from": "NGAP",
      "to": "RRC",
      "message": "NGAP_DOWNLINK_NAS"
    },
    {
      "from": "RRC",
      "to": "NGAP",
      "message": "NGAP_NAS_NON_DELIVERY_IND",
      "precond": "RRC UE Context Unknown"
    },
    {
      "from": "NGAP",
      "to": "AMF",
      "message": "NAS NON DELIVERY INDICATION",
      "postcond": "Dropped due to NGAP UE Context check failure"
    }
  ],
  "state_machine": {
    "states": ["IDLE", "CONNECTED", "RELEASED", "UNKNOWN_RRC", "UNKNOWN_NGAP"],
    "transitions": [
      { "from": "RELEASED", "to": "UNKNOWN_RRC", "on": "Receive DOWNLINK NAS", "guard": "Context Missing in RRC" },
      { "from": "UNKNOWN_RRC", "to": "UNKNOWN_NGAP", "on": "Send Non Delivery Ind", "guard": "Context Missing in NGAP" }
    ]
  },
  "assumptions": ["UE Context is released in both RRC and NGAP, but NGAP receives the initial message."],
  "questions": ["Why does ngap_gNB_handle_nas_downlink allow forwarding if context is missing or if it thinks it exists?"],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "Non-Delivery Indication Reporting",
      "result": "FAIL",
      "evidence": ["ngap_gNB_nas_procedures.c:490 - returns -1 if context not found, dropping the indication."]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "The code in ngap_gNB_nas_non_delivery_ind requires the UE context to be present to send the indication. However, the indication is often needed precisely when the context is unknown or lost. The function should be able to send the indication using the IDs provided in the request even if the full context is missing, provided it can resolve the AMF association."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": ["TS 38.413 8.6.4 NAS Non Delivery Indication"],
    "code_citations": ["openair3/NGAP/ngap_gNB_nas_procedures.c:490"],
    "expected_vs_observed": "According to TS 38.413 8.6.4, if the gNB cannot deliver a NAS PDU, it should send a NAS NON DELIVERY INDICATION. Observed behavior is that OAI NGAP layer drops this indication if the UE context is not found locally, preventing the AMF from knowing about the failure."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "The NGAP layer fails to send NAS NON DELIVERY INDICATION when the UE context is missing. It requires the context to exist to process the indication, which contradicts the scenario where the indication is generated because the context is unknown.",
    "citations": {
      "spec": ["TS 38.413 8.6.4"],
      "code": ["openair3/NGAP/ngap_gNB_nas_procedures.c:490"]
    },
    "suggested_repro": ["Establish UE context", "Release UE context", "Send DOWNLINK NAS TRANSPORT from AMF", "Observe local log error but no outgoing NAS NON DELIVERY INDICATION"],
    "risks": ["AMF may retransmit NAS messages unnecessarily", "State mismatch between AMF and RAN"],
    "next_steps": ["Modify ngap_gNB_nas_non_delivery_ind to handle NULL context", "Implement fallback to find AMF association if context is missing (e.g. using other IDs or default association if possible, though AMF ID is required)"]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.